In [41]:
%pip install -q torch --upgrade
%pip install -q foolbox

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.


In [42]:
import os
import time
import copy
import random

import numpy as np

import torch
import torch.nn.functional as F

from torch import nn
from torch import linalg
from torch.optim import SGD, Optimizer
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import StepLR

from torchvision import transforms, datasets, models

import foolbox as fb

import matplotlib.pyplot as plt

from utils_ods_attack import ods_attack
from utils_general import train, test, short_test
from utils_attacks import whitebox_attack, blackbox_attack_steps
from utils_data import get_dataloader

In [ ]:
# reproducibility issues
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.backends.cudnn.deterministic = True

In [ ]:
# !unzip tiny-imagenet-200.zip

In [ ]:
DATA_NAME = 'cifar'

if DATA_NAME == 'imagenet':
    path = 'tiny-imagenet-200/'
    
    data_loader_train = get_dataloader(path, 'train', batch_size=128)
    data_loader_valid = get_dataloader(path, 'valid', batch_size=128)
    data_loader_test = get_dataloader(path, 'test', batch_size=128)
    
    NUM_OF_CLASSES = 200
    
else:
    dataset_cifar_train = datasets.CIFAR100(
        root="./",
        train=True,
        download=True,
        transform=transforms.ToTensor()
    )
    dataset_cifar_valid_test = datasets.CIFAR100(
        root="./",
        train=False,
        download=True,
        transform=transforms.ToTensor()
    )

    dataset_cifar_valid, dataset_cifar_test = torch.utils.data.random_split(dataset_cifar_valid_test, [5000, 5000], 
                                                                            generator=torch.Generator().manual_seed(42))

    data_loader_train = DataLoader(dataset_cifar_train, batch_size=128, shuffle=True)
    data_loader_valid = DataLoader(dataset_cifar_valid, batch_size=128, shuffle=False)
    data_loader_test = DataLoader(dataset_cifar_test, batch_size=128, shuffle=False)
    
    NUM_OF_CLASSES = 100

In [ ]:
MODEL_NAME = 'resnet50' # change name here if you change the model in the following class

class OurResNetModel(nn.Module):
    """
    Small class to initialize ResNet
    
    """
    def __init__(self, pretrained_bool, num_of_classes):
        super().__init__()
        
        self.model_resnet = models.resnet50(pretrained=pretrained_bool) # 18 34 50 101 152
        self.model_resnet.fc = nn.Linear(self.model_resnet.fc.in_features, num_of_classes, bias=False)

    def forward(self, x):
        return self.model_resnet(x)

In [ ]:
#!g1.1
# optimzer and scheduler params
learning_rate_, weight_decay_ = 0.01, 0.001
step_size_, gamma_ = 3, 0.7

# level of noise, and ODS params
noise_sd_, nu_, num_of_steps_ = 0.1, 0.1, 1

In [ ]:
#!g1.1
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

model = OurResNetModel(True, NUM_OF_CLASSES)
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = SGD(model.parameters(), lr=learning_rate_, momentum=0.0, weight_decay=weight_decay_)
scheduler = StepLR(optimizer, step_size=step_size_, gamma=gamma_)

valid_loss_, valid_acc_ = 0, 0
for epoch in range(10):
    print(f'Epoch {epoch + 1}\n')
    train_loss, train_acc = train(data_loader_train, model, criterion, optimizer, device=device,
                                  augment=False, noise_sd=noise_sd_, attack=False, nu=nu_, num_of_steps=num_of_steps_) # ; print()
    valid_loss, valid_acc = test(data_loader_valid, model, criterion, device=device,
                                 augment=False, noise_sd=noise_sd_, attack=False, nu=nu_, num_of_steps=num_of_steps_); print()
    scheduler.step()
    
    if valid_acc > valid_acc_ * 1.01:
        valid_acc_ = valid_acc
        torch.save({
            'epoch': epoch + 1,
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict(),
        }, os.path.join(f'checkpoint_{DATA_NAME}_{MODEL_NAME}_baseline_eps03.pth.tar'))
    else:
        print("Early stopping")
        checkpoint = torch.load(f'checkpoint_{DATA_NAME}_{MODEL_NAME}_baseline_eps03.pth.tar')
        model.load_state_dict(checkpoint['state_dict'])
        break

model.to(torch.device('cpu')); print()

In [ ]:
#!g1.1
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

print(f'Accuracy on train set: {short_test(data_loader_train, model, device=device)}%')
print(f'Accuracy on valid set: {short_test(data_loader_valid, model, device=device)}%')
print(f'Accuracy on test  set: {short_test(data_loader_test, model, device=device)}%')

In [ ]:
#!g1.1
del model, criterion, optimizer, scheduler

In [ ]:
#!g1.1
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

model = OurResNetModel(True, NUM_OF_CLASSES)
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = SGD(model.parameters(), lr=learning_rate_, momentum=0.0, weight_decay=weight_decay_)
scheduler = StepLR(optimizer, step_size=step_size_, gamma=gamma_)

valid_loss_, valid_acc_ = 0, 0
for epoch in range(10):
    print(f'Epoch {epoch + 1}\n')
    train_loss, train_acc = train(data_loader_train, model, criterion, optimizer, device=device,
                                  augment=True, noise_sd=noise_sd_, attack=False, nu=nu_, num_of_steps=num_of_steps_) # ; print()
    valid_loss, valid_acc = test(data_loader_valid, model, criterion, device=device,
                                 augment=True, noise_sd=noise_sd_, attack=False, nu=nu_, num_of_steps=num_of_steps_); print()
    scheduler.step()
    
    if valid_acc > valid_acc_ * 1.01:
        valid_acc_ = valid_acc
        torch.save({
            'epoch': epoch + 1,
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict(),
        }, os.path.join(f'checkpoint_{DATA_NAME}_{MODEL_NAME}_pixeldp_eps03.pth.tar'))
    else:
        print("Early stopping")
        checkpoint = torch.load(f'checkpoint_{DATA_NAME}_{MODEL_NAME}_pixeldp_eps03.pth.tar')
        model.load_state_dict(checkpoint['state_dict'])
        break

model.to(torch.device('cpu')); print()

In [ ]:
#!g1.1
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

print(f'Accuracy on train set: {short_test(data_loader_train, model, device=device)}%')
print(f'Accuracy on valid set: {short_test(data_loader_valid, model, device=device)}%')
print(f'Accuracy on test  set: {short_test(data_loader_test, model, device=device)}%')

In [ ]:
#!g1.1
del model, criterion, optimizer, scheduler

In [ ]:
#!g1.1
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

model = OurResNetModel(True, NUM_OF_CLASSES)
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = SGD(model.parameters(), lr=learning_rate_, momentum=0.0, weight_decay=weight_decay_)
scheduler = StepLR(optimizer, step_size=step_size_, gamma=gamma_)

valid_loss_, valid_acc_ = 0, 0
for epoch in range(10):
    print(f'Epoch {epoch + 1}\n')
    train_loss, train_acc = train(data_loader_train, model, criterion, optimizer, device=device,
                                  augment=True, noise_sd=noise_sd_, attack=True, nu=nu_, num_of_steps=num_of_steps_) # ; print()
    valid_loss, valid_acc = test(data_loader_valid, model, criterion, device=device,
                                 augment=True, noise_sd=noise_sd_, attack=True, nu=nu_, num_of_steps=num_of_steps_); print()
    scheduler.step()
    
    if valid_acc > valid_acc_ * 1.01:
        valid_acc_ = valid_acc
        torch.save({
            'epoch': epoch + 1,
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict(),
        }, os.path.join(f'checkpoint_{DATA_NAME}_{MODEL_NAME}_ods_eps03.pth.tar'))
    else:
        print("Early stopping")
        checkpoint = torch.load(f'checkpoint_{DATA_NAME}_{MODEL_NAME}_ods_eps03.pth.tar')
        model.load_state_dict(checkpoint['state_dict'])
        break

model.to(torch.device('cpu')); print()

In [ ]:
#!g1.1
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

print(f'Accuracy on train set: {short_test(data_loader_train, model, device=device)}%')
print(f'Accuracy on valid set: {short_test(data_loader_valid, model, device=device)}%')
print(f'Accuracy on test  set: {short_test(data_loader_test, model, device=device)}%')

In [ ]:
#!g1.1
del model, criterion, optimizer, scheduler

In [ ]:
#!g1.1
